In [ ]:
!python -m detectron2.utils.collect_env

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Training on the VisDrone dataset

In [ ]:
# Import PyTorch and TorchVision
import torch, torchvision

# Import some common libraries
import numpy as np
import os, json, cv2, random

# Import some common detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import (
            COCOEvaluator, DatasetEvaluators
            )

In [ ]:
detectron2.utils.collect_env

In [3]:
OUTPUT_DIRECTORY="output/ResNet50FPN_VisDrone"  # The output directory to save logs and checkpoints
CONFIG_FILE_PATH="COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"  # The detectron2 config file for R50-FPN Faster-RCNN
VisDrone_DATASET_PATH="/home/tooba.sheikh/Courses/AI702/VisDrone_FasterRCNN/VisDrone_New"  # Path to iSAID dataset

# Create the output directory if not exists already
if not os.path.exists(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

In [ ]:
setup_logger(f"{OUTPUT_DIRECTORY}/log.txt")

In [5]:
def register_datase(path):
    register_coco_instances("VisDrone2019-DET-train_final", {},
                            f"{path}/VisDrone2019-DET-train/train_coco1.json",
                            f"{path}/VisDrone2019-DET-train/images/")
    register_coco_instances("VisDrone2019-DET-val_final", {},
                            f"{path}/VisDrone2019-DET-val/val_coco1.json",
                            f"{path}/VisDrone2019-DET-val/images/")

register_datase(VisDrone_DATASET_PATH)

In [6]:
def prepare_config(config_path, **kwargs):
    # Parse the expected key-word arguments
    output_path = kwargs["output_dir"]
    workers = kwargs["workers"]

    # Create and initialize the config
    cfg = get_cfg()
    cfg.SEED = 26911042  # Fix the random seed to improve consistency across different runs
    cfg.OUTPUT_DIR = output_path
    cfg.merge_from_file(model_zoo.get_config_file(config_path))
    cfg.DATASETS.TRAIN = ("VisDrone2019-DET-train_final",)
    cfg.DATASETS.TEST = ("VisDrone2019-DET-val_final",)
    cfg.DATALOADER.NUM_WORKERS = workers
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
    # Training schedule - equivalent to 0.25x schedule as per Detectron2 criteria
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.01
    cfg.SOLVER.STEPS = (60000, 80000)
    cfg.SOLVER.MAX_ITER = 161775
    # cfg.SOLVER.MAX_ITER = 28029
    # cfg.SOLVER.MAX_ITER = 90000


    return cfg

d2_config = prepare_config(CONFIG_FILE_PATH, output_dir=OUTPUT_DIRECTORY, workers=2)

In [7]:
# The trainer will automatically perform validation after training
class Trainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return DatasetEvaluators([COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)])

In [8]:
from distutils.version import LooseVersion

In [ ]:
trainer = Trainer(d2_config)  # Create Trainer
trainer.resume_or_load(resume=True)  # Set resume=True if intended to automatically resume training
!export CUDA_LAUNCH_BLOCKING=1
trainer.train()  # Train and evaluate the model

In [ ]:
# Path to the trained ".pth" model file
MODEL_CHECKPOINTS_PATH=f"{OUTPUT_DIRECTORY}/model_final.pth"
d2_config.MODEL.WEIGHTS = MODEL_CHECKPOINTS_PATH  # Update the weights path in the config
d2_config.OUTPUT_DIR = f"{OUTPUT_DIRECTORY}/validate"  # Update the output directory path

setup_logger(f"{d2_config.OUTPUT_DIR}/log.txt")  # Setup the logger
model = Trainer.build_model(d2_config)  # Build model using Trainer
DetectionCheckpointer(model, save_dir=d2_config.OUTPUT_DIR).load(MODEL_CHECKPOINTS_PATH)  # Load the checkpoints
Trainer.test(d2_config, model)  # Test the model on the validation set

In [11]:
def register_test_datase(path):
    register_coco_instances("VisDrone2019-DET-test_final", {},
                            f"{path}/VisDrone2019-DET-test-dev/test_coco1.json",
                            f"{path}/VisDrone2019-DET-test-dev/images")

register_test_datase(VisDrone_DATASET_PATH)

In [ ]:
# Path to the trained ".pth" model file
MODEL_CHECKPOINTS_PATH=f"{OUTPUT_DIRECTORY}/model_final.pth"
d2_config.MODEL.WEIGHTS = MODEL_CHECKPOINTS_PATH  # Update the weights path in the config
d2_config.DATASETS.TEST = ("VisDrone2019-DET-test_final",)  # Set the test set as "iSAID_test"
d2_config.OUTPUT_DIR = f"{OUTPUT_DIRECTORY}/test"  # Update the output directory path

setup_logger(f"{d2_config.OUTPUT_DIR}/log.txt")  # Setup the logger
model = Trainer.build_model(d2_config)  # Build model using Trainer
DetectionCheckpointer(model, save_dir=d2_config.OUTPUT_DIR).load(MODEL_CHECKPOINTS_PATH)  # Load the checkpoints
Trainer.test(d2_config, model)  # Test the model on the validation set